In [10]:
import pandas as pd
import numpy as np
import time

VEHICLE_DATAFILE = 'vehicle_data.csv'

# get df and clean up
vehicle_df = pd.read_csv(
    VEHICLE_DATAFILE,
    parse_dates=['event_datetime'],
    infer_datetime_format=True
).dropna()

# group by vin
vehicle_df = vehicle_df.groupby(['vin'])
supply_df = pd.DataFrame()


def collapse_is_available_events(group):
    global supply_df
    group = group.sort_values(by='event_datetime')
    
    # get event_datetime when is_available goes from true to false (becomes unavailable)
    left = group[(group['is_available'] == False) & (group['is_available'].shift() == True)]
        
    # get event_datetime when is_available goes from false to true (becomes available)
    right = group[(group['is_available'] == True) & (group['is_available'].shift() == False)]
    right['available_at'] = right['event_datetime']  # keep this so we know when it was made available

    # can't assume symmetry for events
    # can't tell which event comes first
    group = pd.merge_asof(left, right, on='event_datetime')
    
    supply_df = supply_df.append(group)

vehicle_df.apply(collapse_is_available_events)
supply_df = supply_df.dropna()
supply_df['unavailable_at'] = supply_df['event_datetime']
supply_df.drop(['event_datetime'], axis=1)
supply_df.reset_index(inplace=True)
supply_df['idle_duration'] = supply_df['unavailable_at'] - supply_df['available_at']  # duration for analysis
supply_df['idle_duration_minutes'] = supply_df['idle_duration'].dt.total_seconds()/60.0
# construct multi-index columns?
supply_df


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


index            available_at  battery_level_x  battery_level_y  \
0          1 2019-04-03 20:57:20.267       100.000000       100.000000   
1          2 2019-04-03 22:12:08.695       100.000000       100.000000   
2          3 2019-04-04 16:25:00.531       100.000000        99.166667   
3          4 2019-04-04 16:25:57.159       100.000000        97.500000   
4          5 2019-04-04 21:26:34.942       100.000000       100.000000   
5          6 2019-04-05 10:08:17.190       100.000000       100.000000   
6          7 2019-04-05 19:08:52.616       100.000000       100.000000   
7          8 2019-04-07 14:25:19.753       100.000000       100.000000   
8          9 2019-04-08 18:32:38.648       100.000000       100.000000   
9         10 2019-04-09 19:43:56.987       100.000000       100.000000   
10        11 2019-04-10 00:00:45.169       100.000000       100.000000   
11        12 2019-04-10 17:05:12.731       100.000000       100.000000   
12        13 2019-04-10 18:28:10.731       100.000000       100.000000   
13        14 2019-04-10 20:51:32.458       100.000000       100.000000   
14        15 2019-04-10 23:35:43.989       100.000000       100.000000   
15        16 2019-04-15 04:53:45.463       100.000000       100.000000   
16        17 2019-04-15 14:53:45.985        99.166667        97.500000   
17        18 2019-04-15 16:01:34.312       100.000000       100.000000   
18        19 2019-04-15 23:56:40.769       100.000000       100.000000   
19        20 2019-04-16 00:01:50.658       100.000000       100.000000   
20        21 2019-04-16 14:00:58.272        98.333333       100.000000   
21        22 2019-04-16 15:34:39.259       100.000000        99.166667   
22        23 2019-04-16 18:56:02.135       100.000000       100.000000   
23        24 2019-04-17 02:16:09.152       100.000000        98.333333   
24        25 2019-04-17 04:13:41.028       100.000000       100.000000   
25        26 2019-04-17 17:49:09.339       100.000000        97.500000   
26        27 2019-04-20 03:27:21.749       100.000000       100.000000   
27        28 2019-04-22 19:36:13.020       100.000000       100.000000   
28        29 2019-04-23 13:58:10.319       100.000000       100.000000   
29        30 2019-04-23 16:26:53.909       100.000000       100.000000   
...      ...                     ...              ...              ...   
50960     33 2019-04-15 05:43:44.956       100.000000       100.000000   
50961     34 2019-04-16 05:52:15.782       100.000000       100.000000   
50962     35 2019-04-17 06:10:51.167       100.000000       100.000000   
50963     36 2019-04-18 05:03:48.049       100.000000       100.000000   
50964     37 2019-04-18 05:39:18.701       100.000000       100.000000   
50965     38 2019-04-18 14:47:36.905       100.000000       100.000000   
50966     39 2019-04-18 15:20:35.714       100.000000       100.000000   
50967     40 2019-04-18 16:10:41.742       100.000000       100.000000   
50968     41 2019-04-18 18:23:53.169       100.000000       100.000000   
50969     42 2019-04-18 19:21:03.084       100.000000       100.000000   
50970     43 2019-04-18 20:35:43.707       100.000000       100.000000   
50971     44 2019-04-18 23:06:08.096       100.000000       100.000000   
50972     45 2019-04-19 00:01:52.819       100.000000       100.000000   
50973     46 2019-04-19 01:07:06.936       100.000000       100.000000   
50974     47 2019-04-19 01:35:59.327       100.000000       100.000000   
50975     48 2019-04-19 03:34:18.455       100.000000       100.000000   
50976     49 2019-04-19 05:55:20.140       100.000000       100.000000   
50977     50 2019-04-19 06:33:51.729       100.000000       100.000000   
50978     51 2019-04-19 07:35:56.421       100.000000       100.000000   
50979     52 2019-04-19 19:09:11.124       100.000000       100.000000   
50980     53 2019-04-19 22:58:58.658       100.000000       100.000000   
50981     54 2019-04-20 21:53:58.098       100.000000       100.000000   
50982     5

In [4]:
# save the intermediate
import time
csv_name = 'supply_df_v6'
supply_df.to_csv(csv_name)

In [6]:
import pandas as pd

csv_name = 'supply_df_v6'

# load the intermediate
try:
    supply_df
except NameError:
    supply_df = pd.read_csv(
        csv_name,
        parse_dates=['event_datetime'],
        infer_datetime_format=True
    ).dropna()

In [ ]:
df = supply_df.apply(
    lambda x: (pd.date_range(x['available_at'], x['unavailable_at'], freq='H', closed='left')).append(pd.to_datetime([x['unavailable_at']])), axis=1)
df

In [ ]:
# construct large dow/hour df
# NOTE: very expensive. should save intermediates so don't have to regenerate
import pandas as pd
import calendar
from copy import deepcopy

# create multi-index and multi-index dataframe
mi = pd.MultiIndex.from_product([list(calendar.day_name), list(range(0, 24))], names=['dow', 'hour'])
base_series = pd.Series(index=mi).fillna(value=0)
mi_df = pd.DataFrame(columns=mi)

def extractor(x):
    global mi_df
    temp = deepcopy(base_series)
    # duration less than 1 hour, does span across slice (hour) ex: [1:30, 2:15]
    if x.size == 2 and x[0].hour != x[1].hour:  
        temp[x[0].day_name(), x[0].hour] += 60 - x[0].minute
        temp[x[1].day_name(), x[1].hour] += x[1].minute

    # duration less than 1 hour, doesn't span across slice (hour) ex: [1:30, 1:45]
    elif x.size == 2 and x[0].hour == x[1].hour:
        temp[x[0].day_name(), x[0].hour] += x[1].minute - x[0].minute
  
    # duration greater than 1 hour, does span across slice (hour) ex: [1:30, 2:30, 2:45]
    elif x.size == 3 and x[1].hour == x[2].hour:
        temp[x[0].day_name(), x[0].hour] += 60 - x[0].minute
        temp[x[2].day_name(), x[2].hour] += x[2].minute
  
    else:
        # duration greater than 2 hours, ex: [1:30, 2:30, 3:30, 3:45]
        # or spans across multiple hours
        n = 0
        min_marker = x[0].minute
        for i, j, k in zip(x.day_name(), x.hour, x.minute):
            # each datetimeindex
            if n == 0: # first element => 60 - 30 = 30
                temp[i, j] += (60 - k)
            elif n == (x.size - 1):  # last element, can't assume full hour
                if k >= min_marker:
                    temp[i, j] += (k - min_marker) # ex: 3:45 - 3:30 = 15m
                else:
                    temp[i, j] += k  # ex: 3:30 - 3:00 = 30m
            elif n == (x.size - 2):  # second to last element, can't assume full hour
                temp[i, j] += k  # ex: 3:30 - 3:00 = 30m
            else:  # middle of array
                temp[i, j] += 60 # ex: 3:30 - 2:30 = 1h
            n += 1
    mi_df = mi_df.append(temp, ignore_index=True)

df.apply(extractor)
mi_csv_name = 'mi_df_v6'
mi_df.to_csv(mi_csv_name)
mi_df

In [3]:
import pandas as pd

mi_csv_name = 'mi_df_v6'

# get the intermediate
try:
    mi_df
except NameError:
    mi_df = pd.read_csv(
        mi_csv_name,
        infer_datetime_format=True
    ).dropna()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# merge the big dow/hour mask back with vehicle_update data

supply_df_with_mask = supply_df.merge(mi_df, left_index=True, right_index=True)

supply_df

Unnamed: 0  index             available_at  battery_level_x  \
0               0      1  2019-04-03 20:57:20.267       100.000000   
1               1      2  2019-04-03 22:12:08.695       100.000000   
2               2      3  2019-04-04 16:25:00.531       100.000000   
3               3      4  2019-04-04 16:25:57.159       100.000000   
4               4      5  2019-04-04 21:26:34.942       100.000000   
5               5      6  2019-04-05 10:08:17.190       100.000000   
6               6      7  2019-04-05 19:08:52.616       100.000000   
7               7      8  2019-04-07 14:25:19.753       100.000000   
8               8      9  2019-04-08 18:32:38.648       100.000000   
9               9     10  2019-04-09 19:43:56.987       100.000000   
10             10     11  2019-04-10 00:00:45.169       100.000000   
11             11     12  2019-04-10 17:05:12.731       100.000000   
12             12     13  2019-04-10 18:28:10.731       100.000000   
13             13     14  2019-04-10 20:51:32.458       100.000000   
14             14     15  2019-04-10 23:35:43.989       100.000000   
15             15     16  2019-04-15 04:53:45.463       100.000000   
16             16     17  2019-04-15 14:53:45.985        99.166667   
17             17     18  2019-04-15 16:01:34.312       100.000000   
18             18     19  2019-04-15 23:56:40.769       100.000000   
19             19     20  2019-04-16 00:01:50.658       100.000000   
20             20     21  2019-04-16 14:00:58.272        98.333333   
21             21     22  2019-04-16 15:34:39.259       100.000000   
22             22     23  2019-04-16 18:56:02.135       100.000000   
23             23     24  2019-04-17 02:16:09.152       100.000000   
24             24     25  2019-04-17 04:13:41.028       100.000000   
25             25     26  2019-04-17 17:49:09.339       100.000000   
26             26     27  2019-04-20 03:27:21.749       100.000000   
27             27     28  2019-04-22 19:36:13.020       100.000000   
28             28     29  2019-04-23 13:58:10.319       100.000000   
29             29     30  2019-04-23 16:26:53.909       100.000000   
...           ...    ...                      ...              ...   
50960       50960     33  2019-04-15 05:43:44.956       100.000000   
50961       50961     34  2019-04-16 05:52:15.782       100.000000   
50962       50962     35  2019-04-17 06:10:51.167       100.000000   
50963       50963     36  2019-04-18 05:03:48.049       100.000000   
50964       50964     37  2019-04-18 05:39:18.701       100.000000   
50965       50965     38  2019-04-18 14:47:36.905       100.000000   
50966       50966     39  2019-04-18 15:20:35.714       100.000000   
50967       50967     40  2019-04-18 16:10:41.742       100.000000   
50968       50968     41  2019-04-18 18:23:53.169       100.000000   
50969       50969     42  2019-04-18 19:21:03.084       100.000000   
50970       50970     43  2019-04-18 20:35:43.707       100.000000   
50971       50971     44  2019-04-18 23:06:08.096       100.000000   
50972       50972     45  2019-04-19 00:01:52.819       100.000000   
50973       50973     46  2019-04-19 01:07:06.936       100.000000   
50974       50974     47  2019-04-19 01:35:59.327       100.000000   
50975       50975     48  2019-04-19 03:34:18.455       100.000000   
50976       50976     49  2019-04-19 05:55:20.140       100.000000   
50977       50977     50  2019-04-19 06:33:51.729       100.000000   
50978       50978     51  2019-04-19 07:35:56.421       100.000000   
50979       50979     52  2019-04-19 19:09:11.124       100.000000   
50980       50980     53  2019-04-19 22:58:58.658       100.000000   
50981       50981     54  2019-04-20 21:53:58.098       100.000000   
50982       50982     55  2019-04-22 14:45:24.276       100.000000   
50983       50983     56  2019-04-22 21:21:55.643       100.000000   
50984       50984     57  2019-04-23 13:50:39.179       100.000000   
50985       50985

In [8]:
supply_mi_merged_csv_name = 'supply_mi_merged_v6'
supply_df_with_mask.to_csv(supply_mi_merged_csv_name)

In [9]:
from pyproj import Proj

supply_mi_merged_csv_name = 'supply_mi_merged_v6'

def convert_to_mercator(lngs, lats):
    projection = Proj(init='epsg:3857')
    xs = []
    ys = []
    for lng, lat in zip(lngs, lats):
        x, y = projection(lng, lat)
        xs.append(x)
        ys.append(y)
    return xs, ys

# convert all points to mercator projection
# get the intermediate
try:
    supply_df_with_mask
except NameError:
    supply_df_with_mask = pd.read_csv(
        supply_mi_merged_csv_name,
        parse_dates=['event_datetime'],
        infer_datetime_format=True
    ).dropna()

supply_df_with_mask['merc_lng'], supply_df_with_mask['merc_lat'] = convert_to_mercator(
    supply_df_with_mask['lng_x'], supply_df_with_mask['lat_x'])

# Start filtering for data to display
# 3 hour chunks
start_hour = 0
end_hour = 3
dow = 'Tuesday'
# filter for 1 hour, 1 dow, this will be the columndatasource later

# filter for the hour range
# figure_df = supply_df[supply_df[dow, hour] != 0]
supply_df_with_mask['analysis_hours'] = supply_df_with_mask[dow, 0] + supply_df_with_mask[dow, 1] + supply_df_with_mask[dow, 2]
figure_df = supply_df_with_mask[supply_df_with_mask['analysis_hours'] != 0]
figure_df  # to be used by figure

KeyError: ('Tuesday', 0)

In [7]:
# create df from selected start and end hours
# bin the rental start positions
import pandas as pd
from pyproj import Proj

# get the rentals file
RENTAL_DATAFILE = 'rental_data_april_2019.csv'

# get df and clean up
rental_df = pd.read_csv(
    RENTAL_DATAFILE,
    parse_dates=['reserved_at', 'booked_at', 'ended_at'],
    infer_datetime_format=True
).dropna()

# extract the rental start dow/hour
rental_df['reserved_at_hour'] = rental_df['reserved_at'].dt.hour
rental_df['reserved_at_dow'] = rental_df['reserved_at'].dt.day_name()

def convert_to_mercator(lngs, lats):
    projection = Proj(init='epsg:3857')
    xs = []
    ys = []
    for lng, lat in zip(lngs, lats):
        x, y = projection(lng, lat)
        xs.append(x)
        ys.append(y)
    return xs, ys

anaylsis_dow = 'Tuesday'
analysis_hour_range = list(range(0, 3))

# convert all points to mercator projection
rental_df['start_merc_lng'], rental_df['start_merc_lat'] = convert_to_mercator(
    rental_df['start_location_lng'], rental_df['start_location_lat'])

rental_df = rental_df.groupby(['reserved_at_dow', 'reserved_at_hour'])
rental_analysis_df = rental_df.get_group((anaylsis_dow, analysis_hour_range[0])).append(
    [rental_df.get_group((anaylsis_dow, analysis_hour_range[1])), 
     rental_df.get_group((anaylsis_dow, analysis_hour_range[2]))])
rental_analysis_df

rental_id  customer_id                reserved_at  \
13        617286        62074 2019-04-30 00:24:52.655560   
328       571923        46439 2019-04-02 00:06:20.558885   
401       582358        64463 2019-04-09 00:09:02.182974   
429       604831        34969 2019-04-23 00:23:09.324836   
628       593769         6893 2019-04-16 00:31:34.897723   
638       617354        26253 2019-04-30 00:58:39.509430   
716       572006        12487 2019-04-02 00:58:43.655977   
834       617319         8232 2019-04-30 00:36:30.073461   
871       582369         5818 2019-04-09 00:17:44.344047   
872       582343        46388 2019-04-09 00:04:49.324744   
892       593779        65738 2019-04-16 00:35:34.311603   
1147      571932        53606 2019-04-02 00:17:20.712188   
1193      593778        58572 2019-04-16 00:34:49.054520   
1243      593787        38331 2019-04-16 00:37:56.465266   
1276      571994        33801 2019-04-02 00:51:49.512301   
1287      571995        48783 2019-04-02 00:53:03.160107   
1372      604872        33584 2019-04-23 00:46:41.254123   
1430      617243        21582 2019-04-30 00:05:28.272714   
1480      593704         9509 2019-04-16 00:05:30.932549   
1536      571953         9389 2019-04-02 00:31:30.333457   
1680      617251        48747 2019-04-30 00:08:12.392342   
1945      617270        44044 2019-04-30 00:14:27.751984   
1956      617257        67420 2019-04-30 00:10:34.982819   
1964      617302        16263 2019-04-30 00:31:03.645450   
1994      593794        40921 2019-04-16 00:42:21.145776   
2097      604878        59446 2019-04-23 00:48:51.258673   
2194      582348        12597 2019-04-09 00:05:40.746841   
2230      617268        58573 2019-04-30 00:13:58.671180   
2236      571924        41515 2019-04-02 00:07:13.942323   
2237      617307        64611 2019-04-30 00:31:53.526910   
...          ...          ...                        ...   
30170     605069        51242 2019-04-23 02:57:33.179297   
30339     617452        52318 2019-04-30 02:04:36.686801   
30411     572137         9085 2019-04-02 02:35:02.532991   
30615     605009        44859 2019-04-23 02:20:51.983834   
30687     617522        32690 2019-04-30 02:59:46.514409   
30723     594024        38168 2019-04-16 02:51:37.186500   
30903     582544        54842 2019-04-09 02:09:34.040841   
31192     572138        50416 2019-04-02 02:35:10.547974   
31196     582582        34583 2019-04-09 02:40:06.996624   
31323     593992        53936 2019-04-16 02:30:39.594556   
31467     572099         6999 2019-04-02 02:04:19.977713   
31571     605049        30293 2019-04-23 02:47:11.583647   
31628     605036        38099 2019-04-23 02:39:40.671096   
31753     617478        57826 2019-04-30 02:26:04.067462   
32020     605053        64866 2019-04-23 02:48:50.990098   
32025     604993        14646 2019-04-23 02:08:30.428445   
32054     617462        32165 2019-04-30 02:11:52.759289   
32165     605058        67678 2019-04-23 02:52:07.398810   
32185     593983        30678 2019-04-16 02:25:15.220075   
32289     582552        43532 2019-04-09 02:13:11.817623   
32302     617484        63119 2019-04-30 02:31:29.416638   
32314     617451        56680 2019-04-30 02:03:14.665502   
32729     605066        26551 2019-04-23 02:56:26.455644   
32742     582602        63103 2019-04-09 02:57:47.658062   
32852     605024         6531 2019-04-23 02:28:07.201275   
33038     572128        34583 2019-04-02 02:27:37.075288   
33159     582540        21482 2019-04-09 02:06:25.662667   
33162     617469        67688 2019-04-30 02:19:36.107622   
33223     617514        27134 2019-04-30 02:57:09.298422   
33434     605039        57583 2019-04-23 02:40:52.363718   

                       booked_at                   ended_at  \
13    2019-04-30 00:25:31.475729 2019-04-30 00:50:25.932545   
328   2019-04-02 00:06:49.143873 2019-04-02 00:28:03.195584   
401   2019-04-09 00:11:36.180662 2019-04-09 05:41:48.035763   
429   2019-04-23 00:33:43.354068 2

In [8]:
# map df to axial (hexes), then take sum of hours 
from bokeh.util.hex import cartesian_to_axial

# binning time
# map the points to hex grid
rental_analysis_df['q'], rental_analysis_df['r'] = cartesian_to_axial(
    rental_analysis_df['start_merc_lng'],
    rental_analysis_df['start_merc_lat'],
    size=500,
    orientation='pointytop'
)

rental_analysis_df

rental_id  customer_id                reserved_at  \
13        617286        62074 2019-04-30 00:24:52.655560   
328       571923        46439 2019-04-02 00:06:20.558885   
401       582358        64463 2019-04-09 00:09:02.182974   
429       604831        34969 2019-04-23 00:23:09.324836   
628       593769         6893 2019-04-16 00:31:34.897723   
638       617354        26253 2019-04-30 00:58:39.509430   
716       572006        12487 2019-04-02 00:58:43.655977   
834       617319         8232 2019-04-30 00:36:30.073461   
871       582369         5818 2019-04-09 00:17:44.344047   
872       582343        46388 2019-04-09 00:04:49.324744   
892       593779        65738 2019-04-16 00:35:34.311603   
1147      571932        53606 2019-04-02 00:17:20.712188   
1193      593778        58572 2019-04-16 00:34:49.054520   
1243      593787        38331 2019-04-16 00:37:56.465266   
1276      571994        33801 2019-04-02 00:51:49.512301   
1287      571995        48783 2019-04-02 00:53:03.160107   
1372      604872        33584 2019-04-23 00:46:41.254123   
1430      617243        21582 2019-04-30 00:05:28.272714   
1480      593704         9509 2019-04-16 00:05:30.932549   
1536      571953         9389 2019-04-02 00:31:30.333457   
1680      617251        48747 2019-04-30 00:08:12.392342   
1945      617270        44044 2019-04-30 00:14:27.751984   
1956      617257        67420 2019-04-30 00:10:34.982819   
1964      617302        16263 2019-04-30 00:31:03.645450   
1994      593794        40921 2019-04-16 00:42:21.145776   
2097      604878        59446 2019-04-23 00:48:51.258673   
2194      582348        12597 2019-04-09 00:05:40.746841   
2230      617268        58573 2019-04-30 00:13:58.671180   
2236      571924        41515 2019-04-02 00:07:13.942323   
2237      617307        64611 2019-04-30 00:31:53.526910   
...          ...          ...                        ...   
30170     605069        51242 2019-04-23 02:57:33.179297   
30339     617452        52318 2019-04-30 02:04:36.686801   
30411     572137         9085 2019-04-02 02:35:02.532991   
30615     605009        44859 2019-04-23 02:20:51.983834   
30687     617522        32690 2019-04-30 02:59:46.514409   
30723     594024        38168 2019-04-16 02:51:37.186500   
30903     582544        54842 2019-04-09 02:09:34.040841   
31192     572138        50416 2019-04-02 02:35:10.547974   
31196     582582        34583 2019-04-09 02:40:06.996624   
31323     593992        53936 2019-04-16 02:30:39.594556   
31467     572099         6999 2019-04-02 02:04:19.977713   
31571     605049        30293 2019-04-23 02:47:11.583647   
31628     605036        38099 2019-04-23 02:39:40.671096   
31753     617478        57826 2019-04-30 02:26:04.067462   
32020     605053        64866 2019-04-23 02:48:50.990098   
32025     604993        14646 2019-04-23 02:08:30.428445   
32054     617462        32165 2019-04-30 02:11:52.759289   
32165     605058        67678 2019-04-23 02:52:07.398810   
32185     593983        30678 2019-04-16 02:25:15.220075   
32289     582552        43532 2019-04-09 02:13:11.817623   
32302     617484        63119 2019-04-30 02:31:29.416638   
32314     617451        56680 2019-04-30 02:03:14.665502   
32729     605066        26551 2019-04-23 02:56:26.455644   
32742     582602        63103 2019-04-09 02:57:47.658062   
32852     605024         6531 2019-04-23 02:28:07.201275   
33038     572128        34583 2019-04-02 02:27:37.075288   
33159     582540        21482 2019-04-09 02:06:25.662667   
33162     617469        67688 2019-04-30 02:19:36.107622   
33223     617514        27134 2019-04-30 02:57:09.298422   
33434     605039        57583 2019-04-23 02:40:52.363718   

                       booked_at                   ended_at  \
13    2019-04-30 00:25:31.475729 2019-04-30 00:50:25.932545   
328   2019-04-02 00:06:49.143873 2019-04-02 00:28:03.195584   
401   2019-04-09 00:11:36.180662 2019-04-09 05:41:48.035763   
429   2019-04-23 00:33:43.354068 2

In [22]:
rental_analysis_df[(rental_analysis_df['q'] == -12681) & (rental_analysis_df['r'] == -6070)]

rental_id  customer_id                reserved_at  \
7017      593781         3596 2019-04-16 00:35:47.562259   
9612      617306        36299 2019-04-30 00:31:52.239608   
11329     593691         8852 2019-04-16 00:00:00.488788   
28455     593717        14883 2019-04-16 00:08:58.278304   
28776     582362        37946 2019-04-09 00:14:58.212733   
33235     593771         4501 2019-04-16 00:31:58.200453   
11608     572078         2866 2019-04-02 01:46:11.378155   

                       booked_at                   ended_at  \
7017  2019-04-16 00:42:49.958184 2019-04-16 00:54:25.415498   
9612  2019-04-30 00:43:19.090438 2019-04-30 01:58:36.531404   
11329 2019-04-16 00:10:19.510909 2019-04-16 01:10:04.912240   
28455 2019-04-16 00:14:32.773954 2019-04-16 00:35:41.777708   
28776 2019-04-09 00:42:02.267983 2019-04-09 01:11:10.533836   
33235 2019-04-16 00:38:16.752980 2019-04-16 00:52:52.786485   
11608 2019-04-02 01:57:17.355245 2019-04-02 02:07:16.326881   

       start_location_lat  start_location_lng  end_location_lat  \
7017            37.806810          -122.26499         37.805637   
9612            37.808746          -122.26682         37.760727   
11329           37.810410          -122.26673         37.829758   
28455           37.807175          -122.26509         37.867430   
28776           37.810730          -122.26735         37.860546   
33235           37.811157          -122.26755         37.846066   
11608           37.811237          -122.26226         37.795834   

       end_location_lng  reserved_at_hour reserved_at_dow  start_merc_lng  \
7017         -122.24762                 0         Tuesday   -1.361048e+07   
9612         -122.25883                 0         Tuesday   -1.361068e+07   
11329        -122.26344                 0         Tuesday   -1.361067e+07   
28455        -122.28136                 0         Tuesday   -1.361049e+07   
28776        -122.27239                 0         Tuesday   -1.361074e+07   
33235        -122.27834                 0         Tuesday   -1.361076e+07   
11608        -122.27051                 1         Tuesday   -1.361017e+07   

       start_merc_lat      q     r  
7017     4.552170e+06 -12681 -6070  
9612     4.552443e+06 -12681 -6070  
11329    4.552678e+06 -12681 -6070  
28455    4.552222e+06 -12681 -6070  
28776    4.552723e+06 -12681 -6070  
33235    4.552783e+06 -12681 -6070  
11608    4.552794e+06 -12681 -6070

In [21]:
# map df to axial (hexes), then take sum of hours 
from bokeh.util.hex import cartesian_to_axial

# binning time
# map the points to hex grid
figure_df['q'], figure_df['r'] = cartesian_to_axial(
    figure_df['merc_lng'],
    figure_df['merc_lat'],
    size=500,
    orientation='pointytop'
)

11122    JTDKDTB30G1138230
16876    JTDKDTB31J1605629
22644    JTDKDTB33H1592313
30616    JTDKDTB35J1605990
47133    JTDKDTB39J1606592
49056    JTDKDTB3XH1592633
Name: vin_y, dtype: object
11122   2019-04-30 00:15:28.554
16876   2019-04-15 16:14:51.684
22644   2019-04-15 16:08:51.320
30616   2019-04-16 00:13:48.375
47133   2019-04-08 21:55:49.610
49056   2019-04-02 00:54:12.154
Name: available_at, dtype: datetime64[ns]
11122   2019-04-30 00:31:52.281
16876   2019-04-16 00:08:58.325
22644   2019-04-16 00:35:47.612
30616   2019-04-16 00:31:58.254
47133   2019-04-09 00:14:58.257
49056   2019-04-02 01:46:11.429
Name: unavailable_at, dtype: datetime64[ns]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [14]:
# binned_df[('Monday', 10),]  # FYI groupby changes the syntax
binned_df = figure_df.groupby(['q', 'r'])

# could add idle duration data later
idle_minutes_df = binned_df['idle_duration_minutes'].agg(['count', 'mean', 'median', 'sum', 'min', 'max']).reset_index()

idle_minutes_df.drop(idle_minutes_df.columns[[0, 1, 2]], axis=1, inplace=True)
idle_minutes_df.columns = ['total_idle_mins_mean', 'total_idle_mins_median',
                           'total_idle_mins_sum', 'total_idle_mins_min', 'total_idle_mins_max']

# binned_df[('Monday', 10),]  # FYI groupby changes the syntax. access like this
_hexbin_source = binned_df['analysis_hours'].agg(['count', 'mean', 'median', 'sum', 'min', 'max']).reset_index()
_hexbin_source.columns = ['q', 'r', 'idle_events_count', 'idle_minutes_per_hour_mean',
                          'idle_minutes_per_hour_median', 'idle_minutes_per_hour_sum',
                          'idle_minutes_per_hour_min', 'idle_minutes_per_hour_max']
_hexbin_source = _hexbin_source.merge(idle_minutes_df, left_index=True, right_index=True)
_hexbin_source

q     r  idle_events_count  idle_minutes_per_hour_mean  \
0   -12715 -6063                  1                   60.000000   
1   -12714 -6065                  2                  180.000000   
2   -12714 -6064                  2                  180.000000   
3   -12714 -6063                  2                  124.500000   
4   -12713 -6065                  1                  180.000000   
5   -12713 -6064                  1                  180.000000   
6   -12713 -6063                  2                  180.000000   
7   -12712 -6064                  6                  118.333333   
8   -12711 -6065                  4                  180.000000   
9   -12711 -6064                 10                  180.000000   
10  -12711 -6063                  1                  180.000000   
11  -12709 -6051                  3                  180.000000   
12  -12707 -6058                 23                  163.739130   
13  -12706 -6059                  3                  180.000000   
14  -12705 -6062                 25                  119.320000   
15  -12704 -6063                  1                  180.000000   
16  -12702 -6065                 32                  113.937500   
17  -12702 -6064                 27                  133.925926   
18  -12701 -6068                 11                  165.272727   
19  -12701 -6065                  9                  119.777778   
20  -12701 -6064                 25                  171.640000   
21  -12700 -6070                 10                  164.200000   
22  -12700 -6068                 53                  125.415094   
23  -12698 -6099                  4                  171.500000   
24  -12697 -5999                  1                  180.000000   
25  -12694 -5996                  1                    1.000000   
26  -12689 -6066                  1                  180.000000   
27  -12688 -6066                  3                  156.000000   
28  -12688 -6065                  3                   54.666667   
29  -12688 -6063                  2                   65.500000   
..     ...   ...                ...                         ...   
331 -12505 -6215                 13                  117.692308   
332 -12505 -6214                 16                  162.812500   
333 -12505 -6213                  4                  150.750000   
334 -12505 -6212                 12                  110.166667   
335 -12505 -6211                  7                  121.857143   
336 -12505 -6210                  4                  166.750000   
337 -12505 -6209                 11                  117.090909   
338 -12505 -6208                  5                  172.800000   
339 -12504 -6214                  4                   84.750000   
340 -12504 -6213                  9                  161.222222   
341 -12504 -6212                  9                  127.444444   
342 -12504 -6211                  4                   37.500000   
343 -12504 -6210                  6                   88.833333   
344 -12504 -6209                  8                  121.125000   
345 -12503 -6214                  1                  180.000000   
346 -12503 -6213                  4                  150.000000   
347 -12503 -6212                  4                  180.000000   
348 -12503 -6211                  7                  170.285714   
349 -12503 -6210                 10                   97.300000   
350 -12503 -6209                 10                  147.800000   
351 -12503 -6203                  1                  180.000000   
352 -12502 -6205                216                  170.254630   
353 -12502 -6204                157                  179.248408   
354 -12501 -6210                  1                    1.000000   
355 -12497 -6222                  1                  180.000000   
356 -12486 -6222                  1                  180.000000   
357 -12462 -6248                  1                  180.000000   
358 -12430 -6579                  1                  180

In [41]:
# join rental df to _hexbin_source df on q and r
# create new column of rentals/vehicles
_hexbin_source = _hexbin_source.merge(rental_hexbin, on=['q', 'r'])
_hexbin_source['rental_vehicle_ratio'] = _hexbin_source['counts']/_hexbin_source['idle_events_count']
_hexbin_source

q     r  idle_events_count  idle_minutes_per_hour_mean  \
0   -12714 -6063                  2                  124.500000   
1   -12707 -6058                 23                  163.739130   
2   -12705 -6062                 25                  119.320000   
3   -12702 -6065                 32                  113.937500   
4   -12702 -6064                 27                  133.925926   
5   -12701 -6065                  9                  119.777778   
6   -12701 -6064                 25                  171.640000   
7   -12700 -6070                 10                  164.200000   
8   -12700 -6068                 53                  125.415094   
9   -12694 -5996                  1                    1.000000   
10  -12688 -6066                  3                  156.000000   
11  -12688 -6065                  3                   54.666667   
12  -12687 -6066                  2                  164.000000   
13  -12687 -6065                  1                   51.000000   
14  -12687 -6063                  4                   58.500000   
15  -12686 -6070                  7                  121.714286   
16  -12686 -6069                  5                   32.200000   
17  -12686 -6066                  3                   72.000000   
18  -12686 -6063                  7                   96.714286   
19  -12686 -6061                  1                   93.000000   
20  -12685 -6071                  9                   78.222222   
21  -12685 -6070                 10                   75.100000   
22  -12685 -6069                  5                   40.600000   
23  -12685 -6066                  3                  103.000000   
24  -12685 -6065                  3                   84.666667   
25  -12685 -6064                  1                   35.000000   
26  -12685 -6063                  4                   69.000000   
27  -12685 -6060                  5                  113.000000   
28  -12684 -6071                 18                  116.000000   
29  -12684 -6070                  7                   52.714286   
..     ...   ...                ...                         ...   
201 -12508 -6215                 33                  128.939394   
202 -12508 -6214                 29                  103.862069   
203 -12508 -6213                 27                  141.444444   
204 -12508 -6212                  6                  136.833333   
205 -12508 -6211                  5                  115.200000   
206 -12508 -6210                  8                  133.625000   
207 -12507 -6218                  3                   74.333333   
208 -12507 -6217                 16                  114.750000   
209 -12507 -6215                 43                  134.139535   
210 -12507 -6214                 11                   80.454545   
211 -12507 -6213                 23                  143.956522   
212 -12507 -6211                  2                  159.500000   
213 -12506 -6216                 13                  133.230769   
214 -12506 -6215                 22                  127.818182   
215 -12506 -6214                 21                  133.761905   
216 -12506 -6213                 15                  100.200000   
217 -12506 -6209                  3                   67.666667   
218 -12506 -6208                  7                  129.571429   
219 -12505 -6214                 16                  162.812500   
220 -12505 -6213                  4                  150.750000   
221 -12505 -6212                 12                  110.166667   
222 -12505 -6211                  7                  121.857143   
223 -12504 -6214                  4                   84.750000   
224 -12504 -6211                  4                   37.500000   
225 -12504 -6210                  6                   88.833333   
226 -12504 -6209                  8                  121.125000   
227 -12503 -6213                  4                  150.000000   
228 -12503 -6210                 10                   97

In [50]:
from bokeh.plotting import figure, show, output_file
from bokeh.transform import linear_cmap
from bokeh.tile_providers import CARTODBPOSITRON

# draw the map
map_figure = figure(
    x_range=(-13618976.4221, -13605638.1607),  # bounding box for starting view
    y_range=(4549035.0828, 4564284.2700),
    x_axis_type='mercator',
    y_axis_type='mercator',
    plot_width=750,
    plot_height=750,
    title=f'Available Supply for {dow} {start_hour} to {end_hour}',
    tooltips=[('(q, r)', '(@q, @r)'),
              ('vehicles', '@idle_events_count'),
              ('rentals', '@counts'),
              ('rental_vehicle_ratio', '@rental_vehicle_ratio'),
              ('mean idle minutes/hour', '@idle_minutes_per_hour_mean'),
              ('median idle minutes/hour', '@idle_minutes_per_hour_median'),
              ('sum idle minutes/hour', '@idle_minutes_per_hour_sum'),
              ('min idle minutes/hour', '@idle_minutes_per_hour_min'),
              ('max idle minutes/hour', '@idle_minutes_per_hour_max'),
              ('mean total idle minutes', '@total_idle_mins_mean'),
              ('median total idle minutes', '@total_idle_mins_median'),
              ('sum total idle minutes', '@total_idle_mins_sum'),
              ('min total idle minutes', '@total_idle_mins_min'),
              ('max total idle minutes', '@total_idle_mins_max')]
)

# add background of streets for context
map_figure.add_tile(CARTODBPOSITRON)

# add hexes, with modified fill color based on mean of idle time per hour
map_figure.hex_tile(q='q', r='r', size=500, source=_hexbin_source,
                    hover_color='pink', hover_alpha=0.8, fill_alpha=0.3,
                    fill_color=linear_cmap('rental_vehicle_ratio', 'Magma256',
                                           0, max(_hexbin_source['rental_vehicle_ratio'])))

GlyphRenderer(id='1430', ...)

In [51]:
# generate map in browser
from bokeh.io import show
show(map_figure)